In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
from selenium import webdriver
import pandas as pd
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from matplotlib import pyplot as plt

In [2]:
from selenium import webdriver

# Set up the Selenium WebDriver (you'll need to install ChromeDriver or other browser drivers)
driver = webdriver.Chrome()
driver.maximize_window()
url = "https://results.eci.gov.in/AcResultGenOct2024/index.htm"
driver.get(url)
driver.implicitly_wait(3)
#haryana = driver.find_element(By.XPATH,"//h2[@class='state-title'  and text()='Haryana']/parent::*" )
haryana=driver.find_element(By.XPATH,"//h2[@class='state-title'  and contains(text(),'Jammu')]/parent::*")
haryana.click()

In [3]:
# Scroll the page
driver.implicitly_wait(3)
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Wait for new content to load
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:  # Break if no more content is loaded
        break
    last_height = new_height


In [5]:
btn=driver.find_element(By.XPATH,"//a[contains(text(), 'Constituencies')]")
btn.click()

In [7]:
#  Fecth the header of the table

headers_elements =driver.find_elements(By.XPATH,"//th[contains(text(), 'Constituencies')]/parent::tr/following-sibling::tr/child::th")
headers=[]
for h in headers_elements:
    #print(h.text)
    headers.append(h.text)

In [8]:
# total pages 
pages = driver.find_element(By.XPATH,'//ul[@class="pagination pagination-sm"]').text
pages_lst = re.findall(r'\d',pages)
pages_lst

['1', '2', '3', '4', '5']

In [13]:
# This function will fetch the details of candidtes from the table and returns a list of lists for a 'single page'
def get_candiates_list():
    elements = driver.find_elements(By.XPATH,"//tfoot/preceding-sibling::tbody/tr")
    candidates=[]
    for ele in elements:
        candidates_info=[]
        for e in ele.find_elements(By.XPATH,'./td'):
            candidates_info.append(e.text)
        candidates.append(candidates_info)
    return candidates

In [15]:
candidates_data=[]

In [17]:
candidates_data = get_candiates_list()

In [19]:
#candidates_data

In [18]:
for i in pages_lst[1:]:
    try:    
        # Attempt to find the hyperlink
        hyperlink = driver.find_element(By.XPATH, "//li[@class='page-item active']/following-sibling::li[1]/a")
        # Get the URL from the hyperlink
        url = hyperlink.get_attribute("href")
        # Print the URL
        print(url)
        driver.get(url)
        candidates_data.extend(get_candiates_list())
    except NoSuchElementException:
        print("The hyperlink was not found.")

https://results.eci.gov.in/AcResultGenOct2024/statewiseU082.htm
https://results.eci.gov.in/AcResultGenOct2024/statewiseU083.htm
https://results.eci.gov.in/AcResultGenOct2024/statewiseU084.htm
https://results.eci.gov.in/AcResultGenOct2024/statewiseU085.htm


In [20]:
len(candidates_data)

90

In [23]:
df = pd.DataFrame(data=candidates_data,columns = headers)

In [25]:
df.head()

,Constituency,Const. No.,Leading Candidate,Leading Party,Trailing Candidate,Trailing Party,Margin,Round,Status
0,AKHNOOR(SC),81,MOHAN LAL,Bharatiya Janata Party\ni,ASHOK KUMAR,Indian National Congress\ni,24679,15/15,Result Declared
1,ANANTNAG,44,PEERZADA MOHAMMAD SYED,Indian National Congress\ni,MEHBOOB BEG,Jammu & Kashmir Peoples Democratic Party\ni,1686,9/9,Result Declared
2,ANANTNAG WEST,43,ABDUL MAJEED BHAT,Jammu & Kashmir National Conference\ni,ABDUL GAFFAR SOFI,Jammu & Kashmir Peoples Democratic Party\ni,10435,14/14,Result Declared
3,BAHU,75,VIKRAM RANDHAWA,Bharatiya Janata Party\ni,TARANJIT SINGH TONY,Indian National Congress\ni,11251,18/18,Result Declared
4,BANDIPORA,15,NIZAM UDDIN BHAT,Indian National Congress\ni,USMAN ABDUL MAJID,Independent\ni,811,12/12,Result Declared


In [29]:
df.columns

Index(['Constituency', 'Const. No.', 'Leading Candidate', 'Leading Party',
       'Trailing Candidate', 'Trailing Party', 'Margin', 'Round', 'Status'],
      dtype='object')

In [41]:
columns = []
for col in df.columns:
    col = col.lower().replace(' ','_').replace('.','')
    columns.append(col)

In [43]:
df.columns= columns

In [45]:
df.sample()

,constituency,const_no,leading_candidate,leading_party,trailing_candidate,trailing_party,margin,round,status
68,RAMGARH(SC),69,DR DEVINDER KUMAR MANYAL,Bharatiya Janata Party\ni,YASH PAUL KUNDAL,Indian National Congress\ni,14202,10/10,Result Declared


In [47]:
for column in df.columns:
   df[column]= df[column].str.lower()

In [49]:
df.sample(5)

,constituency,const_no,leading_candidate,leading_party,trailing_candidate,trailing_party,margin,round,status
59,pampore,32,hasnain masoodi,jammu & kashmir national conference\ni,zahoor ahmad mir,jammu & kashmir peoples democratic party\ni,2763,12/12,result declared
73,shopian,37,shabir ahmad kullay,independent\ni,sheikh mohammad rafi,jammu & kashmir national conference\ni,1207,13/13,result declared
0,akhnoor(sc),81,mohan lal,bharatiya janata party\ni,ashok kumar,indian national congress\ni,24679,15/15,result declared
50,lal chowk,22,sheikh ahsan ahmed,jammu & kashmir national conference\ni,mohammad ashraf mir,jammu and kashmir apni party\ni,11343,13/13,result declared
18,chenani,61,balwant singh mankotia,bharatiya janata party\ni,harsh dev singh,j&k national panthers party (india)\ni,15611,12/12,result declared


In [51]:
df["trailing_party"]=df["trailing_party"].apply(lambda value:value.replace('\ni',''))
df["leading_party"]=df["leading_party"].apply(lambda value:value.replace('\ni',''))

In [53]:
df.sample(5)

,constituency,const_no,leading_candidate,leading_party,trailing_candidate,trailing_party,margin,round,status
8,basohli,65,darshan kumar,bharatiya janata party,ch. lal singh,indian national congress,16034,8/8,result declared
1,anantnag,44,peerzada mohammad syed,indian national congress,mehboob beg,jammu & kashmir peoples democratic party,1686,9/9,result declared
9,beerwah,28,shafi ahmad wani,jammu & kashmir national conference,nazir ahmad khan,independent,4161,16/16,result declared
0,akhnoor(sc),81,mohan lal,bharatiya janata party,ashok kumar,indian national congress,24679,15/15,result declared
6,banihal,55,sajad shaheen,jammu & kashmir national conference,imtiaz ahmed shan,jammu & kashmir peoples democratic party,6110,14/14,result declared


In [55]:
#df[df["trailing_candidate"].apply(lambda value : bool(re.match(r"^(dr\.|mr\.|mrs\.|smt\.)",value)))]

In [57]:
df["const_no"]=df["const_no"].astype(int)
df["margin"]=df["margin"].astype(int)

In [59]:
df["round_a"] = [ lst[0]  for lst in df["round"].apply(lambda value:value.split('/'))]
df["round_b"] = [ lst[1]  for lst in df["round"].apply(lambda value:value.split('/'))]

In [61]:
df.drop(columns=["round","round_b"],inplace=True)

In [63]:
df["round"]=df["round_a"]

In [65]:
df

,constituency,const_no,leading_candidate,leading_party,trailing_candidate,trailing_party,margin,status,round_a,round
0,akhnoor(sc),81,mohan lal,bharatiya janata party,ashok kumar,indian national congress,24679,result declared,15,15
1,anantnag,44,peerzada mohammad syed,indian national congress,mehboob beg,jammu & kashmir peoples democratic party,1686,result declared,9,9
2,anantnag west,43,abdul majeed bhat,jammu & kashmir national conference,abdul gaffar sofi,jammu & kashmir peoples democratic party,10435,result declared,14,14
3,bahu,75,vikram randhawa,bharatiya janata party,taranjit singh tony,indian national congress,11251,result declared,18,18
4,bandipora,15,nizam uddin bhat,indian national congress,usman abdul majid,independent,811,result declared,12,12
...,...,...,...,...,...,...,...,...,...,...
85,uri,9,sajjad shafi,jammu & kashmir national conference,taj mohi ud din,independent,14469,result declared,11,11
86,vijaypur,71,chander parkash,bharatiya janata party,rajesh kumar pargotra,jammu & kashmir national conference,19040,result declared,9,9
87,wagoora - kreeri,12,irfan hafiz lone,indian national congress,syed basharat ahmed bukhari,jammu & kashmir peoples democratic party,7751,result declared,12,12
88,zadibal,24,tanvir sadiq,jammu & kashmir national conference,abid hussain ansari,jammu & kashmir people conference,16173,result declared,12,12


In [67]:
df["constituency"]=df["constituency"].apply(lambda value: value.replace('.',''))

In [69]:
df.sample(5)

,constituency,const_no,leading_candidate,leading_party,trailing_candidate,trailing_party,margin,status,round_a,round
35,inderwal,48,payare lal sharma,independent,ghulam mohd saroori,independent,643,result declared,12,12
63,r s pura- jammu south,74,dr. narinder singh raina,bharatiya janata party,raman bhalla,indian national congress,1966,result declared,17,17
42,karnah,1,javaid ahmad mirchal,jammu & kashmir national conference,naseer ahmad awan,jammu & kashmir people conference,6262,result declared,8,8
56,nowshera,84,surinder kumar choudhary,jammu & kashmir national conference,ravinder raina,bharatiya janata party,7819,result declared,10,10
75,sonawari,14,hilal akbar lone,jammu & kashmir national conference,yasir reshi,independent,13744,result declared,12,12


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   constituency        90 non-null     object
 1   const_no            90 non-null     int32 
 2   leading_candidate   90 non-null     object
 3   leading_party       90 non-null     object
 4   trailing_candidate  90 non-null     object
 5   trailing_party      90 non-null     object
 6   margin              90 non-null     int32 
 7   status              90 non-null     object
 8   round_a             90 non-null     object
 9   round               90 non-null     object
dtypes: int32(2), object(8)
memory usage: 6.5+ KB


In [73]:
df["round"]=df["round"].astype(int)

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   constituency        90 non-null     object
 1   const_no            90 non-null     int32 
 2   leading_candidate   90 non-null     object
 3   leading_party       90 non-null     object
 4   trailing_candidate  90 non-null     object
 5   trailing_party      90 non-null     object
 6   margin              90 non-null     int32 
 7   status              90 non-null     object
 8   round_a             90 non-null     object
 9   round               90 non-null     int32 
dtypes: int32(3), object(7)
memory usage: 6.1+ KB


In [77]:
df.drop(columns='round_a',inplace=True)

In [79]:
df

,constituency,const_no,leading_candidate,leading_party,trailing_candidate,trailing_party,margin,status,round
0,akhnoor(sc),81,mohan lal,bharatiya janata party,ashok kumar,indian national congress,24679,result declared,15
1,anantnag,44,peerzada mohammad syed,indian national congress,mehboob beg,jammu & kashmir peoples democratic party,1686,result declared,9
2,anantnag west,43,abdul majeed bhat,jammu & kashmir national conference,abdul gaffar sofi,jammu & kashmir peoples democratic party,10435,result declared,14
3,bahu,75,vikram randhawa,bharatiya janata party,taranjit singh tony,indian national congress,11251,result declared,18
4,bandipora,15,nizam uddin bhat,indian national congress,usman abdul majid,independent,811,result declared,12
...,...,...,...,...,...,...,...,...,...
85,uri,9,sajjad shafi,jammu & kashmir national conference,taj mohi ud din,independent,14469,result declared,11
86,vijaypur,71,chander parkash,bharatiya janata party,rajesh kumar pargotra,jammu & kashmir national conference,19040,result declared,9
87,wagoora - kreeri,12,irfan hafiz lone,indian national congress,syed basharat ahmed bukhari,jammu & kashmir peoples democratic party,7751,result declared,12
88,zadibal,24,tanvir sadiq,jammu & kashmir national conference,abid hussain ansari,jammu & kashmir people conference,16173,result declared,12


In [81]:
df["leading_party"].value_counts()

leading_party
jammu & kashmir national conference         42
bharatiya janata party                      29
independent                                  7
indian national congress                     6
jammu & kashmir peoples democratic party     3
aam aadmi party                              1
jammu & kashmir people conference            1
communist party of india (marxist)           1
Name: count, dtype: int64

In [83]:
#Majority 
Majority  = int(df.shape[0]/2)+1
df_total_cnt=df["leading_party"].value_counts()

In [85]:
df_total_cnt[df_total_cnt>=Majority]

Series([], Name: count, dtype: int64)

In [89]:
df[["constituency","round","leading_party","leading_candidate"]][df["round"]==df["round"].max()].sort_values("leading_party")

,constituency,round,leading_party,leading_candidate
61,poonch haveli,24,jammu & kashmir national conference,ajaz ahmed jan


In [91]:
#Seat Percentage
df.shape[0]

90

In [93]:
leading_party_values = df["leading_party"].value_counts().values/df.shape[0]*100
leading_party_values=leading_party_values.round(2)
leading_party_values

array([46.67, 32.22,  7.78,  6.67,  3.33,  1.11,  1.11,  1.11])

In [95]:
df["leading_party"].value_counts().index

Index(['jammu & kashmir national conference', 'bharatiya janata party',
       'independent', 'indian national congress',
       'jammu & kashmir peoples democratic party', 'aam aadmi party',
       'jammu & kashmir people conference',
       'communist party of india (marxist)'],
      dtype='object', name='leading_party')

In [97]:
pd.DataFrame({
"party":df["leading_party"].value_counts().index,
 "Seat_share_%":leading_party_values
    
}
            )

,party,Seat_share_%
0,jammu & kashmir national conference,46.67
1,bharatiya janata party,32.22
2,independent,7.78
3,indian national congress,6.67
4,jammu & kashmir peoples democratic party,3.33
5,aam aadmi party,1.11
6,jammu & kashmir people conference,1.11
7,communist party of india (marxist),1.11


In [99]:
df["margin"]

0     24679
1      1686
2     10435
3     11251
4       811
      ...  
85    14469
86    19040
87     7751
88    16173
89    13233
Name: margin, Length: 90, dtype: int32

In [79]:
var = input('Enter Your Age and Nationality:')

Enter Your Age and Nationality: 18 Indian


 var.split()


'Indian'

In [86]:
if 18<=int(var.split()[0]) and ("").join(var.split()[1:]).lower()=='indian':
    print(" You're Eligible to Vote")
else:
    print("You're Not Eligible")


 You're Eligible to Vote


In [126]:
if 5 == True:
    print('pass')
else:
    print('fail')

fail
